In [1]:
# Step 0: Import Necessary Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
import os
import zipfile
import random
import numpy as np

In [2]:
# Step 1: Set Seed for Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Step 2: Load and Prepare the Dataset
zip_file_path = "/content/drive/MyDrive/archive.zip"  # Path to your zip file
extracted_data_dir = "/content/ASL_PROJECT"  # Directory to extract to

# Unzip the dataset if not already extracted
if not os.path.exists(extracted_data_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_data_dir)

data_dir = extracted_data_dir

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "/content/ASL_PROJECT/ASL_Alphabet_Dataset/asl_alphabet_train"

datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize pixel values
    validation_split=0.2   # 20% for validation
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),  # Resize images to 64x64
    batch_size=32,
    class_mode="categorical",  # Multi-class classification
    subset="training"
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

print(f"Number of classes (train): {train_data.num_classes}")


In [ ]:
# Step 4: Define the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()


In [ ]:
# Step 5: Train the Model with Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    "best_sign_language_model.keras",  # Use .keras for compatibility
    save_best_only=True,
    monitor='val_loss'
)

start_time = time.time()  # Start timer
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Adjust if necessary
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)
end_time = time.time()  # End timer
runtime = end_time - start_time


In [ ]:
# Step 6: Evaluate the Model
test_loss, test_acc = model.evaluate(val_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Training Runtime: {runtime:.2f} seconds")

In [ ]:
# Step 7: Save the Model
model.save("final_sign_language_model.h5")


In [ ]:
# Step 8: Plot Accuracy and Loss
plt.figure(figsize=(8, 4))
# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()